<a href="https://colab.research.google.com/github/samehra/Projects/blob/master/generation/langchain/handbook/create_urls_sec_edgar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sec-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from sec_api import QueryApi

queryApi = QueryApi(api_key="YOUR_API_KEY")

In [4]:
query = {
  "query": { "query_string": { 
      "query": "formType:\"10-K\" " + 
               "AND NOT formType:\"NT 10-K\" " + 
               "AND NOT formType:\"10-K/A\" " +
               "AND filedAt:[2022-01-01 TO 2022-12-31]",
      "time_zone": "America/New_York"
  } },
  "from": "0",
  "size": "10",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

#response = queryApi.get_filings(query)

In [8]:
# open the file we use to store the filing URLs
log_file = open("filing_urls.txt", "a")

for from_batch in range(0, 30, 10):
  # set new "from" starting position of search 
  #base_query["from"] = from_batch;
  query["from"] = from_batch;
  response = queryApi.get_filings(query)

  # no more filings in search universe
  if len(response["filings"]) == 0:
    break;

  # for each filing, only save the URL pointing to the filing itself 
  # and ignore all other data. 
  # the URL is set in the dict key "linkToFilingDetails"
  urls_list = list(map(lambda x: x["linkToFilingDetails"], response["filings"]))

  # transform list of URLs into one string by joining all list elements
  # and add a new-line character between each element.
  urls_string = "\n".join(urls_list) + "\n"
  
  log_file.write(urls_string)
  
log_file.close()

print("All URLs downloaded")

All URLs downloaded


In [14]:
from sec_api import ExtractorApi

extractorApi = ExtractorApi("YOUR_API_KEY")

In [21]:
# download filing and save to "filings" folder
def download_filing(filing_url):

  items = ["1", "6", "7", "7A"]

  for item in items:
    print("item:", item, "url", filing_url)

    try:
      section_text = extractorApi.get_section(filing_url=filing_url, 
                                              section=item, 
                                              return_type="text")

      # file_name example: 000156459019027952-msft-10k_20190630.htm
      file_name = filing_url.split("/")[-3] + "-" + filing_url.split("/")[-2] + "-" + filing_url.split("/")[-1] 
      download_to = "./filings/" + file_name
      with open(download_to, "w") as f:
        f.write(section_text + "\n")

    except Exception as e:
      print("Problem with {url}".format(filing_url=filing_url))
      print(e)

In [22]:
# load URLs from log file
def load_urls():
  log_file = open("filing_urls.txt", "r")
  urls = log_file.read().split("\n") # convert long string of URLs into a list 
  log_file.close()
  return urls

In [23]:
urls[0]

'https://www.sec.gov/Archives/edgar/data/1374881/000147793222009583/king_10k.htm'

In [24]:
import os
import multiprocessing

def download_all_filings():
  print("Start downloading all filings")

  download_folder = "./filings" 
  if not os.path.isdir(download_folder):
    os.makedirs(download_folder)
    
  # uncomment next line to process all URLs
  # urls = load_urls()
  #urls = load_urls()[0:40]
  urls = load_urls()[0]
  print("{length} filing URLs loaded".format(length=len(urls)))

  number_of_processes = 20

  with multiprocessing.Pool(number_of_processes) as pool:
    pool.map(download_filing, urls)
  
  print("All filings downloaded")

In [25]:
download_all_filings()

Start downloading all filings
79 filing URLs loaded
item:item:item:item:item:   item:  111  item: item:item:item:urlitem:urlitem:item:item:11item:item:item: item:item:item:url        1        1 1  11t/11url1 11 url111 11:  
   
 url     urlurlurl url 
url wurlurl urlurlurlurl  purl url  
url hv     s 
.e  w
s.gc

w
o/

t/








item:item: 1item:   item:11url item: item:  item:item:1urlurlA      
item:  1111urlcr1 
    
 urlurlhurlurlurl
     evsi/




item: 1 url eitem:
item: item:1   1url1 url  url a
 item:g
d 1
 urlitem: r 
1item: 1  urlurl  d/

item: 1 url a
item:item: 1  1item:url   1urlt
  urlitem: a/ 

1item: item:  item:1url1    1url 1 urlurl 
 743


item: 1 url 8
item: 1 url 8
item:item:  1 url /
1 item:url 1
item: item:item:1 url 1   101
   url 0
url url 10

item: 1 url 4
item: 71 item:url  
1item:item:  url1   1url 9item:  
7url
1  url3 
item:item: item:  1211
 url  url url  202


item: 1 url 0
item: 1item:  item:url1 item:1  item:url 9url 
item:  18 1 
 51 urlurl 
 /
 url 

KeyError: ignored

Let's inspect the Query API response and print a subset of properties of each filing, namely `formType` and `periodOfReport`. The filings live in the `response["filings"]` list. 

Feel free to skip the next lines if you're familiar with the use of `map` and `lambda`. The `map` function applies the `lambda` function to every filing in the list of filings. The `lambda` function simply returns a new dict for each filing by extracting the `formType` and `periodOfReport` property from the filing, and setting both values as the values of the new dict. Finally, the result of the `map` function is converted into a new list. 

In [ ]:
list(map(lambda x: {"formType": x["formType"], "periodOfReport": x["periodOfReport"]}, response["filings"]))

[{'formType': '10-K', 'periodOfReport': '2021-11-30'},
 {'formType': '10-K', 'periodOfReport': '2021-10-31'},
 {'formType': '10-K', 'periodOfReport': '2021-10-31'},
 {'formType': '10-K', 'periodOfReport': '2021-09-30'},
 {'formType': '10-K', 'periodOfReport': '2021-09-30'},
 {'formType': '10-K', 'periodOfReport': '2021-09-30'},
 {'formType': '10-K', 'periodOfReport': '2021-09-30'},
 {'formType': '10-K', 'periodOfReport': '2021-09-30'},
 {'formType': '10-K', 'periodOfReport': '2021-09-30'},
 {'formType': '10-K', 'periodOfReport': '2021-09-30'}]

In [ ]:
period_by_cik = {}
period_by_ticker = {}

for filing in response["filings"]:
  cik, ticker, periodOfReport = filing["cik"], filing["ticker"], filing["periodOfReport"]

  if not cik in period_by_cik:
    period_by_cik[cik] = []

  if not periodOfReport in period_by_cik[cik]:
    period_by_cik[cik].append(periodOfReport)

  if len(ticker) > 0:
    if not ticker in period_by_ticker:
      period_by_ticker[ticker] = []

    if not periodOfReport in period_by_ticker[ticker]:
      period_by_ticker[ticker].append(periodOfReport)

In [ ]:
period_by_cik

{'1206942': ['2021-10-31'],
 '1341726': ['2021-09-30'],
 '1377167': ['2021-09-30'],
 '1435181': ['2021-09-30'],
 '1502966': ['2021-09-30'],
 '1592782': ['2021-09-30'],
 '1725516': ['2021-11-30'],
 '1844817': ['2021-09-30'],
 '1853314': ['2021-09-30'],
 '72633': ['2021-10-31']}

In [ ]:
period_by_ticker

{'AACI': ['2021-09-30'],
 'DIGP': ['2021-09-30'],
 'FGCO': ['2021-09-30'],
 'GIAC': ['2021-09-30'],
 'GSPE': ['2021-09-30'],
 'NBLD': ['2021-11-30'],
 'NRT': ['2021-10-31'],
 'NUKK': ['2021-09-30'],
 'PHBI': ['2021-09-30']}